# Notebook magics
We start a cluster as done in the [Introduction](./intro.ipynb)

In [1]:
import ipyparallel as ipp
cluster = ipp.Cluster(n=4, engines="mpi")
rc = cluster.start_and_connect_sync(activate=True)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

We can now use the %%px magic to use this cluster in a notebook cell

In [5]:
%%px
from mpi4py import MPI
import os
import time
start = time.time()
rank = MPI.COMM_WORLD.rank
pid = os.getpid()
if rank == 0:
    time.sleep(2)
print(f"{time.time()-start:.2e} Hello I am process {rank} with Unix ID {pid}")

[stdout:2] 1.90e-04 Hello I am process 2 with Unix ID 36298


[stdout:3] 2.02e-04 Hello I am process 3 with Unix ID 36299


[stdout:1] 4.17e-04 Hello I am process 1 with Unix ID 36297


%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] 2.00e+00 Hello I am process 0 with Unix ID 36296


You have all variables defined in a %%px cell in the next cell declared with this magic as well 

In [7]:
%%px
print(rank)

[stdout:0] 0


[stdout:1] 1


[stdout:3] 3


[stdout:2] 2


Note that we have not imported `mpi4py` outside of the cluster

In [10]:
try:
    print(MPI.COMM_WORLD.size)
except NameError as e:
    print(f"Error: {e}")

Error: name 'MPI' is not defined
